In [0]:
#--------------------------------------------------------------------------
#   Program name: PngInfo_To_Txt_ForMultiplePng_v01.ipynb
#
#   Purpose:
#       1. Extract the "Positive prompt value" and "Negative prompt value" 
#          from the png files generated by Stable Diffusion Webui.
#       2. Format the "Positive prompt value" and "Negative prompt value" 
#          for using in "Prompts from file or textbox" of Stable Diffusion Webui.
#       3. Output the formatted text in the "*.txt" file.
#       4. Batch process the above steps.
#
#   Usage:
#       1. Put the *.ipynb file in the target folder (where you run the *.ipynb file).
#       2. Create a folder named "PNG files" under the target folder.
#       3. Put the target png files in the folder "PNG files".
#       4. Run all cells in the *.ipynb file. Or use the hot key `Shift + Enter` 
#          to run the program, since just only one cell in the program.
#       5. A file named "png_info_input_01.txt" will be created in the target folder after the process. 
#          It can be used in "Prompts from file or textbox" of Stable Diffusion Webui.
#
#   Suitable version of Stable Diffusion Webui:
#       v1.7.0 (tested on 06 Feb. 2024)
#
#   Other information:
#       Date:   06 Feb. 2024
#       Author: Dr. Hsien-Ching Chung
#       ORCiD:  https://orcid.org/0000-0001-9364-8858
#
#       Project Link: https://github.com/HsienChing/
#
#   License: MIT License
#   Copyright (c) 2024 Hsien-Ching Chung
#--------------------------------------------------------------------------




#----------------------------------------
# Import modules
#----------------------------------------

from os import walk
from os.path import join
from PIL import Image

#----------------------------------------
# Define function()
#----------------------------------------

def PngInfoToTxt(filepath):
    ## Open image file according to filepath.
    img = Image.open(filepath)
    
    ## Read image parameters.
    # The image parameters are stored in the dictionary type variable with length 1.
    # image parameters format example: 
    #     {'parameters': '[Positive prompt value]Negative prompt: [Value]Steps: [Value], 
    #                     Sampler: [Value], CFG scale: [Value], Seed: [Value], Size: [Value], 
    #                     Model hash: [Value], Model: [Value], Denoising strength: [Value], 
    #                     Hires upscale: [Value], Hires upscaler: [Value], Lora hashes: [Value], 
    #                     TI hashes: [Value], Version: [Value]'}
    parameters_dict = img.text  
    # print(parameters_dict)       # output: {'parameters': [Parameter values]}
    # print(type(parameters_dict)) # output: <class 'dict'>
    # print( len(parameters_dict)) # output: 1

    ## Extrac string of parameter values.
    parameter_value = parameters_dict['parameters']
    # print(parameter_value)       # [Parameter values]
    # print(type(parameter_value)) # output: <class 'str'>
    # print( len(parameter_value)) # output: string lenght

    ## Extract string positions.
    Position_Negative_prompt = parameter_value.find('Negative prompt:')
    Position_Steps           = parameter_value.find('Steps:')

    ## Extract the string of the positive prompt.
    Positive_prompt_original = parameter_value[:Position_Negative_prompt]
    # print(Positive_prompt_original)

    ## Eliminate '\n' in the positive prompt string.
    Positive_prompt_oneline  = Positive_prompt_original.replace('\n', '')
    # print(Positive_prompt_oneline)

    ## Extract the string of the negative prompt.
    Negative_prompt_original = parameter_value[(Position_Negative_prompt + len('Negative prompt: ')):Position_Steps]
    # print(Negative_prompt_original)

    ## Eliminate '\n' in the negative prompt string.
    Negative_prompt_oneline  = Negative_prompt_original.replace('\n', '')
    # print(Negative_prompt_oneline)

    ## Format the output text according to Stable Diffusion Webui.
    # REF: Prompts from file or textbox
    #      https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Features
    # Output text format: --prompt "[Positive prompt]" 
    #                     --negative_prompt "[Negative prompt]"
    Output_text = '--prompt "'          + Positive_prompt_oneline + '" ' +\
                  '--negative_prompt "' + Negative_prompt_oneline + '" '
    # print(Output_text)

    ## Return information.
    return(Output_text)

#----------------------------------------
# File I/O
#----------------------------------------

## [Input] Relative path of the folder containing the png files.
# Note: The png files are generated by Stable Diffusion Webui. Otherwise, errors might occur.
path_PngFolder  = './PNG files'

## [Input] Output filename.
output_filename = 'png_info_input_01.txt'

#----------------------------------------
# Main program
#----------------------------------------

## Recursive directory traversing.
for dirpath, dirnames, filenames in walk(path_PngFolder):
    for f in filenames:
        fullpath = join(dirpath, f)
        # print(fullpath)

        ## Use PngInfoToTxt(filepath) function to get the output text.
        Output_text = PngInfoToTxt(fullpath)

        ## Write output text to the output file.
        with open(output_filename, 'a') as output_file:
            output_file.write(Output_text + '\n')